In [ ]:
# -*- coding: utf-8 -*-
"""
對每個 target = (a,b)：
  1) 產生四種曲線 (line, cycloid, parabola, ellipse)
  2) 以通過原點且指向 (a,b) 的直線為鏡像軸，把該 target 的曲線全部鏡像
只顯示鏡像後的曲線，格網為正方形。
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton, brentq

# --------------------------
# 使用者設定（改這裡）
# --------------------------
targets = [(6, 1)]   # list of (a,b)
nx = 600
colors = {'line':'C0','cycloid':'C1','parabola':'C2','ellipse':'C3'}

# --------------------------
# 曲線生成函式
# --------------------------
def find_theta_star(k):
    def f(theta):
        return theta - np.sin(theta) - k*(1 - np.cos(theta))
    ts = np.linspace(1e-6, 2*np.pi-1e-6, 400)
    fs = f(ts)
    idx = np.where(np.sign(fs[:-1]) * np.sign(fs[1:]) <= 0)[0]
    if idx.size > 0:
        left, right = ts[idx[0]], ts[idx[0]+1]
        try:
            return brentq(f, left, right, xtol=1e-12, rtol=1e-12, maxiter=200)
        except Exception:
            pass
    for guess in (1.0,2.0,3.0,4.0):
        try:
            theta_star = newton(f, guess, tol=1e-12, maxiter=200)
            if theta_star > 1e-8:
                return theta_star
        except Exception:
            continue
    return None

def cycloid_xy_scaled(a, b, theta_star, npoints=600):
    R = b / (1 - np.cos(theta_star))
    thetas = np.linspace(0.0, theta_star, npoints)
    x_raw = R * (thetas - np.sin(thetas))
    y_raw = R * (1 - np.cos(thetas))
    sx = 1.0 if x_raw[-1] == 0 else (a / x_raw[-1])
    x = x_raw * sx
    y = y_raw.copy()
    return x, y

def line_xy(a, b, npoints=600):
    xs = np.linspace(0, a, npoints)
    ys = (b / a) * xs if a != 0 else np.zeros_like(xs)
    return xs, ys

def parabola_xy(a, b, npoints=600):
    xs = np.linspace(0, a, npoints)
    with np.errstate(divide='ignore', invalid='ignore'):
        ys = b * np.sqrt(np.clip(xs / a, 0.0, 1.0))
    ys[0] = 0.0
    return xs, ys

def ellipse_xy(a, b, npoints=600):
    thetas = np.linspace(0, np.pi/2, npoints)
    xs = -a * np.cos(thetas) + a
    ys = b * np.sin(thetas)
    return xs, ys

# --------------------------
# 鏡像函式（關於通過原點且方向為 u 的直線）
# --------------------------
def reflect_points_across_line(x_arr, y_arr, u_vec):
    pts = np.vstack([x_arr.ravel(), y_arr.ravel()])   # shape (2, N)
    dot = (u_vec.reshape(2,1).T @ pts)                # shape (1, N)
    reflected = 2 * u_vec.reshape(2,1) @ dot - pts   # shape (2, N)
    xr = reflected[0, :].reshape(x_arr.shape)
    yr = reflected[1, :].reshape(y_arr.shape)
    return xr, yr

# --------------------------
# 為每個 target 生成並鏡像它的曲線；僅保留鏡像結果
# --------------------------
mirrored_curves = []
mirrored_B_points = []

for (a, b) in targets:
    curves_for_target = []
    xl, yl = line_xy(a, b, nx); curves_for_target.append(('Line', xl, yl, colors['line']))
    theta_star = find_theta_star(a / b if b != 0 else np.inf)
    if theta_star is not None:
        try:
            xcy, ycy = cycloid_xy_scaled(a, b, theta_star, nx)
            curves_for_target.append(('Cycloid', xcy, ycy, colors['cycloid']))
        except Exception:
            curves_for_target.append(('Cycloid_approx', np.array([0,a]), np.array([0,b]), colors['cycloid']))
    else:
        curves_for_target.append(('Cycloid_fail', np.array([0,a]), np.array([0,b]), colors['cycloid']))
    xp, yp = parabola_xy(a, b, nx); curves_for_target.append(('Parabola', xp, yp, colors['parabola']))
    xe, ye = ellipse_xy(a, b, nx); curves_for_target.append(('Ellipse', xe, ye, colors['ellipse']))

    # 鏡像軸方向：使用 (a,b) 自身決定（通過原點）
    u = np.array([a, b], dtype=float)
    if np.allclose(u, 0):
        u = np.array([1.0, 0.0])
    u = u / np.linalg.norm(u)

    for name, xr0, yr0, col in curves_for_target:
        xm, ym = reflect_points_across_line(np.asarray(xr0), np.asarray(yr0), u)
        # 若需要可排序以避免繪圖連線順序錯亂，例如： idx = np.argsort(xm); xm, ym = xm[idx], ym[idx]
        mirrored_curves.append({'x': xm, 'y': ym, 'label': f'{name} mirrored to axis->({a},{b})', 'color': col})
    a_m, b_m = reflect_points_across_line(np.array([a]), np.array([b]), u)
    mirrored_B_points.append((a_m[0], b_m[0]))

# --------------------------
# 計算顯示範圍（依鏡像後的曲線）並確保方形 data-aspect
# --------------------------
x_min = min(np.min(c['x']) for c in mirrored_curves)
x_max = max(np.max(c['x']) for c in mirrored_curves)
y_min = min(np.min(c['y']) for c in mirrored_curves)
y_max = max(np.max(c['y']) for c in mirrored_curves)

bm = np.array(mirrored_B_points)
x_min = min(x_min, np.min(bm[:,0]))
x_max = max(x_max, np.max(bm[:,0]))
y_min = min(y_min, np.min(bm[:,1]))
y_max = max(y_max, np.max(bm[:,1]))

x_min = min(x_min, 0.0)
y_min = min(y_min, 0.0)

pad_frac = 0.03
dx = x_max - x_min
dy = y_max - y_min
if dx == 0: dx = max(1.0, abs(x_max)*0.1)
if dy == 0: dy = max(1.0, abs(y_max)*0.1)
pad_x = dx * pad_frac; pad_y = dy * pad_frac
x0, x1 = x_min - pad_x, x_max + pad_x
y0, y1 = y_min - pad_y, y_max + pad_y

plot_w = x1 - x0; plot_h = y1 - y0
if plot_w > plot_h:
    extra = plot_w - plot_h
    y0 -= extra/2; y1 += extra/2
else:
    extra = plot_h - plot_w
    x0 -= extra/2; x1 += extra/2

target_long_inch = 10.0
plot_w = x1 - x0; plot_h = y1 - y0
if plot_w >= plot_h:
    fig_w = target_long_inch; fig_h = max(2.0, target_long_inch * (plot_h / plot_w))
else:
    fig_h = target_long_inch; fig_w = max(2.0, target_long_inch * (plot_w / plot_h))

# --------------------------
# 繪圖（僅鏡像曲線），整數格線並在 B 點標註座標
# --------------------------
fig, ax = plt.subplots(figsize=(fig_w, fig_h))

for c in mirrored_curves:
    ax.plot(c['x'], c['y'], label=c['label'], color=c['color'], linewidth=1.2)

bm_x = [p[0] for p in mirrored_B_points]
bm_y = [p[1] for p in mirrored_B_points]
ax.scatter(bm_x, bm_y, color='k', marker='x', zorder=10, label='Strating point')

# 設定顯示範圍
ax.set_xlim(x0, x1)
ax.set_ylim(y0, y1)

# 整數格線：以整數為 major ticks（從向下取整的 x0/y0 到向上取整的 x1/y1）
x_int0 = int(np.floor(x0))
x_int1 = int(np.ceil(x1))
y_int0 = int(np.floor(y0))
y_int1 = int(np.ceil(y1))

# 產生整數刻度序列（步進為 1）
x_ticks = np.arange(x_int0, x_int1 + 1, 1)
y_ticks = np.arange(y_int0, y_int1 + 1, 1)

ax.set_xticks(x_ticks)
ax.set_yticks(y_ticks)

# 若範圍過大導致刻度擁擠，可以在此改成步進 2 或 5：
# 如果刻度數 > 25，自動改步進
max_ticks = 25
if len(x_ticks) > max_ticks:
    step = int(np.ceil(len(x_ticks) / max_ticks))
    x_ticks = np.arange(x_int0, x_int1 + 1, step)
    ax.set_xticks(x_ticks)
if len(y_ticks) > max_ticks:
    step = int(np.ceil(len(y_ticks) / max_ticks))
    y_ticks = np.arange(y_int0, y_int1 + 1, step)
    ax.set_yticks(y_ticks)

# 方形比例與格線樣式
ax.set_aspect('equal', adjustable='box')
ax.grid(which='major', color='gray', linestyle='-', linewidth=0.6)

# 在每個 B 點旁加座標標註（避免重疊，略微偏移）
for (xm, ym) in mirrored_B_points:
    txt = f"({xm:.2f}, {ym:.2f})"
    # 偏移：向右上方移動一個格子寬度的 0.2
    dx = (x1 - x0) / max(10, len(x_ticks))
    dy = (y1 - y0) / max(10, len(y_ticks))
    ax.text(xm + 0.15*dx, ym + 0.15*dy, txt, fontsize=9, color='k', zorder=20,
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=1.0))

# 畫每個 target 的鏡像軸（原點 → target），以虛線標示
for (a,b), (am,bm) in zip(targets, mirrored_B_points):
    u = np.array([a,b], dtype=float)
    if np.allclose(u,0): continue
    u = u/np.linalg.norm(u)
    axis_len = max(x1 - x0, y1 - y0) * 1.2
    line_pts = np.vstack([np.linspace(-axis_len, axis_len, 3), np.linspace(-axis_len, axis_len, 3)]) * u.reshape(2,1)
    ax.plot(line_pts[0,:], line_pts[1,:], color='k', linestyle='--', linewidth=0.8)

ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('paths (target → origin)')
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize='small')

plt.tight_layout()
plt.show()

分割-----------------
-
-
-
-
-


In [ ]:
# --------------------------
# 修正：以鏡像後的 B 作為起點，計算從 B -> A 的物理時間參數（重力）
# --------------------------
import os

def time_param_from_B_to_A(x, y, g=9.81, eps_h=1e-6):
    """
    輸入節點 (x,y)（通常是從 A->B 的順序），
    回傳 t_norm 與 t_raw，對應於從 B -> A 的時間參數化（起點 B, 終點 A）。
    實作步驟：
      - 先把點反向為從 B -> A
      - 對每段以段中點高度差 (y_start - y_mid) 計算速度 v_mid = sqrt(2 g dh)
      - dt = ds / max(v_mid, tiny) 並累加得到 t_raw（保證單調）
    回傳的 t_norm 與 t_raw 對應於反向後的點順序（B->A）。
    """
    x = np.asarray(x)
    y = np.asarray(y)
    # 如果原始序列是 A->B，反轉為 B->A；若你原先是 B->A，這不會改變
    x_rev = x[::-1]
    y_rev = y[::-1]

    dx = np.diff(x_rev)
    dy = np.diff(y_rev)
    ds = np.hypot(dx, dy)
    if ds.size == 0:
        return np.array([0.0]), np.array([0.0])

    y_start = y_rev[0]                       # 起點為 B 的高度
    # 段中點高度（針對反向序列）
    y_mid = 0.5 * (y_rev[:-1] + y_rev[1:])
    # 高度差（下落量），若為負表示該段在上升（此時速度仍用小正值避免 nan）
    dh = y_start - y_mid
    dh = np.maximum(dh, eps_h)              # 確保非負且避免零
    v_mid = np.sqrt(2.0 * g * dh)           # 中點速度近似
    v_mid = np.maximum(v_mid, 1e-8)         # 防止除零或極小
    dt = ds / v_mid
    t_raw_rev = np.concatenate([[0.0], np.cumsum(dt)])  # 時間從 B 開始累加到 A
    if t_raw_rev[-1] == 0:
        t_norm_rev = t_raw_rev.copy()
    else:
        t_norm_rev = t_raw_rev / t_raw_rev[-1]
    # 返回對應於反向序列（B->A）
    return t_norm_rev, t_raw_rev

def sample_point_by_time_BtoA(x, y, t_norm_rev, t_target_rev):
    """
    對於以 B->A 排序的節點與對應 t_norm_rev，插值取 t_target_rev (0..1) 的點。
    注意：輸入的 x,y 仍給原始（A->B）或任何順序，但我們會在此處把它反向以符合 t_norm_rev。
    """
    x = np.asarray(x)[::-1]   # 反向成 B->A
    y = np.asarray(y)[::-1]
    t_norm = np.asarray(t_norm_rev)
    # 保險處理小尺寸
    if t_norm.size == 1:
        return x[0], y[0]
    t_target = float(t_target_rev)
    if t_target <= t_norm[0]:
        return x[0], y[0]
    if t_target >= t_norm[-1]:
        return x[-1], y[-1]
    ix = np.searchsorted(t_norm, t_target) - 1
    frac = (t_target - t_norm[ix]) / (t_norm[ix+1] - t_norm[ix])
    xm = x[ix] + frac * (x[ix+1] - x[ix])
    ym = y[ix] + frac * (y[ix+1] - y[ix])
    return xm, ym

# 使用者參數：顯示時間與 frames
T = 5.0
n_frames = 30
save_frames = False

# 為每條鏡像曲線預先計算對應的 B->A 時間參數（t_norm_rev 與 t_raw_rev）
for c in mirrored_curves:
    t_norm_rev, t_raw_rev = time_param_from_B_to_A(c['x'], c['y'], g=9.81, eps_h=1e-6)
    c['t_norm_rev'] = t_norm_rev
    c['t_raw_rev'] = t_raw_rev
# 真實時間尺度（取所有曲線的最大真實時間，作為 display 映射參考）
t_ends = [c['t_raw_rev'][-1] for c in mirrored_curves if 't_raw_rev' in c]
t_max_real = max(t_ends) if len(t_ends) > 0 else 1.0

# frames 時間序列（display time）
times_display = np.linspace(0.0, T, n_frames)
out_dir = "frames_output"
if save_frames:
    os.makedirs(out_dir, exist_ok=True)

for i, t_disp in enumerate(times_display):
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    u_global = t_disp / T  # 0..1

    for c in mirrored_curves:
        # 把 display 時間對應到該條曲線的真實時間，再轉為該曲線的正規化 t (B->A)
        t_real = u_global * t_max_real
        t_target_norm = 0.0 if c['t_raw_rev'][-1] <= 0 else np.clip(t_real / c['t_raw_rev'][-1], 0.0, 1.0)
        xm, ym = sample_point_by_time_BtoA(c['x'], c['y'], c['t_norm_rev'], t_target_norm)
        ax.plot(c['x'], c['y'], color=c.get('color', 'C0'), alpha=0.6, linewidth=1.0)
        ax.plot(xm, ym, marker='o', markersize=6, color='red', zorder=20)

    # 畫鏡像後的 B 點
    bm_x = [p[0] for p in mirrored_B_points]; bm_y = [p[1] for p in mirrored_B_points]
    ax.scatter(bm_x, bm_y, color='k', marker='x', zorder=15)

    # 範圍、整數格線、比例與格線（與你主程式一致）
    ax.set_xlim(x0, x1); ax.set_ylim(y0, y1)
    x_int0 = int(np.floor(x0)); x_int1 = int(np.ceil(x1))
    y_int0 = int(np.floor(y0)); y_int1 = int(np.ceil(y1))
    x_ticks = np.arange(x_int0, x_int1 + 1, 1); y_ticks = np.arange(y_int0, y_int1 + 1, 1)
    max_ticks = 25
    if len(x_ticks) > max_ticks:
        step = int(np.ceil(len(x_ticks) / max_ticks)); x_ticks = np.arange(x_int0, x_int1 + 1, step)
    if len(y_ticks) > max_ticks:
        step = int(np.ceil(len(y_ticks) / max_ticks)); y_ticks = np.arange(y_int0, y_int1 + 1, step)
    ax.set_xticks(x_ticks); ax.set_yticks(y_ticks)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(which='major', color='gray', linestyle='-', linewidth=0.5)

    ax.text(0.02, 0.98, f"display t = {t_disp:.2f}/{T:.2f}\n(real t = {u_global * t_max_real:.2f}s)",
            transform=ax.transAxes, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8, pad=3))

    # 畫鏡像軸
    for (a,b), (am,bm) in zip(targets, mirrored_B_points):
        u = np.array([a,b], dtype=float)
        if np.allclose(u,0): continue
        u = u / np.linalg.norm(u)
        axis_len = max(x1 - x0, y1 - y0) * 1.2
        line_pts = np.vstack([np.linspace(-axis_len, axis_len, 3), np.linspace(-axis_len, axis_len, 3)]) * u.reshape(2,1)
        ax.plot(line_pts[0,:], line_pts[1,:], color='k', linestyle='--', linewidth=0.8)

    ax.set_xlabel('x (m)'); ax.set_ylabel('y (m)')
    ax.set_title(f'Mirrored paths (frame {i+1}/{n_frames})')

    plt.tight_layout()
    if save_frames:
        fname = os.path.join(out_dir, f"frame_{i:04d}.png")
        fig.savefig(fname, dpi=150)
    else:
        plt.show()
    plt.close(fig)